In [43]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

In [44]:
load_url = "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=040&bs=040&ta=20&sc=20214&sngz=&po1=25&pc=50"

res = requests.get(load_url)
res.encoding = "utf-8"
soup = BeautifulSoup(res.text, "html.parser")

In [45]:
# ページ数の取得
pages = soup.find("ol", class_="pagination-parts")
num_of_pages = int(pages.find_all("li")[-1].text)

In [46]:
# データフレーム作成
rental_property_datas = pd.DataFrame(columns=["名前", "住所", "最寄り駅１","最寄り駅２","最寄り駅３", "築年数", "建物全体の階数", "階数", "賃料", "管理費", "敷金", " 礼金", "間取り", "面積" ])

In [54]:
for p in range(num_of_pages):

    # topのページは"&page=1"を付けても開ける
    page_url = load_url + "&page=" + str(p+1)
    res = requests.get(page_url)
    res.encoding = "utf-8"
    soup = BeautifulSoup(res.text, "html.parser")
    
    # サーバー負荷を避けるため1s遅延
    time.sleep(1)

    # 「建物ごとに表示」ページから物件毎の情報を取得
    cassetitems = soup.find_all("div", class_="cassetteitem")

    for i in range(len(cassetitems)):
        
        # 上の部分（建物情報）
        details = cassetitems[i].find_all("div", class_="cassetteitem-detail")
        
        for ii in range(len(details)):
            name = details[ii].find("div", class_="cassetteitem_content-title").text
            address = details[ii].find("li", class_="cassetteitem_detail-col1").text
            _stations = details[ii].find_all("div", class_="cassetteitem_detail-text")
            station1 = _stations[0].text
            station2 = _stations[1].text
            station3 = _stations[2].text
            _col3 = details[ii].find_all("li", class_="cassetteitem_detail-col3")
            building_age = _col3[0].find_all("div")[0].text
            number_of_floors = _col3[0].find_all("div")[1].text
            
            # 下の部分（部屋の情報）
            items = cassetitems[i].find("div", class_="cassetteitem-item")
            tbodys = items.find_all("tbody")
            for iii in range(len(tbodys)):
                _tds = tbodys[iii].find_all("td")
                floor = _tds[2].text.replace('\r','').replace('\n','').replace('\t','')
                rent = _tds[3].find("span", class_="cassetteitem_other-emphasis ui-text--bold").text
                maintenance_fee = _tds[3].find("span", class_="cassetteitem_price cassetteitem_price--administration").text
                deposit = _tds[4].find("span", class_="cassetteitem_price cassetteitem_price--deposit").text
                gratuity = _tds[4].find("span", class_="cassetteitem_price cassetteitem_price--gratuity").text
                layout = _tds[5].find("span", class_="cassetteitem_madori").text
                area = _tds[5].find("span", class_="cassetteitem_menseki").text
                
                _d = pd.DataFrame()
                _d["名前"] = [name]
                _d["住所"] = [address]
                _d["最寄り駅１"] = [station1]
                _d["最寄り駅２"] = [station2]
                _d["最寄り駅３"] = [station3]
                _d["築年数"] = [building_age]
                _d["建物全体の階数"] = [number_of_floors]
                _d["階数"] = [floor]
                _d["賃料"] = [rent]
                _d["管理費"] = [maintenance_fee]
                _d["敷金"] = [deposit]
                _d["礼金"] = [gratuity]
                _d["間取り"] = [layout]
                _d["面積"] = [area]

                rental_property_datas = pd.concat([rental_property_datas, _d], ignore_index=True)
                if i == 0 :
                    print(name)
                    print(address)
                    print(station1)
                    print(station2)
                    print(station3)
                    print(building_age)
                    print(number_of_floors)
                    print(floor)
                    print(rent)
                    print(maintenance_fee)
                    print(deposit)
                    print(gratuity)
                    print(layout)
                    print(area)
                
                

ヒルサイドケンタッキー
長野県茅野市ちの横内
ＪＲ中央線/茅野駅 歩13分
ＪＲ中央線/上諏訪駅 歩81分

築25年
2階建
1階
5.3万円
4300円
-
-
2DK
44.34m2
ブルック　ウエスト　II
長野県茅野市宮川
ＪＲ中央線/茅野駅 歩16分
ＪＲ中央線/青柳駅 歩79分

築11年
2階建
1階
6.5万円
3300円
-
9.75万円
1LDK
46.24m2
永和ハイツ
長野県茅野市中大塩
ＪＲ中央線/茅野駅 歩69分


築39年
2階建
1階
3.2万円
-
-
-
2DK
33m2
レジデンストキワＢ棟
長野県茅野市宮川
ＪＲ中央線/茅野駅 歩18分


築33年
2階建
2階
5.5万円
-
-
-
3K
54m2
レジデンストキワＢ棟
長野県茅野市宮川
ＪＲ中央線/茅野駅 歩18分


築33年
2階建
1階
5.5万円
-
-
-
3K
50m2
上原五味ハイツＢ棟
長野県茅野市ちの
ＪＲ中央線/茅野駅 歩24分


築35年
2階建
1階
4.1万円
1500円
-
-
2DK
48.4m2
レオパレスディアコート
長野県茅野市豊平
ＪＲ中央線/茅野駅 バス9分 (バス停)鬼場 歩7分


築22年
2階建
1階
4.1万円
5000円
-
4.1万円
1K
23.18m2
レオパレスディアコート
長野県茅野市豊平
ＪＲ中央線/茅野駅 バス9分 (バス停)鬼場 歩7分


築22年
2階建
2階
4.2万円
5000円
-
4.2万円
1K
23.18m2


In [53]:
rental_property_datas

,名前,住所,最寄り駅１,最寄り駅２,最寄り駅３,築年数,建物全体の階数,階数,賃料,管理費,敷金,礼金,間取り,面積,礼金
0,ヒルサイドケンタッキー,長野県茅野市ちの横内,ＪＲ中央線/茅野駅 歩13分,ＪＲ中央線/上諏訪駅 歩81分,,築25年,2階建,\r1階,5.3万円,4300円,-,NaN,2DK,44.34m2,-
1,リバーサイドスクウェアＡ,長野県茅野市豊平,ＪＲ中央線/茅野駅 歩54分,,,築23年,2階建,\r1階,4.4万円,5300円,-,NaN,2DK,40.04m2,-
2,エスポワール２１Ｃ,長野県茅野市塚原１,ＪＲ中央線/茅野駅 歩9分,,,築24年,2階建,\r1階,5.35万円,4700円,-,NaN,2DK,44.34m2,-
3,ＪＲ中央線 茅野駅 2階建 築15年,長野県茅野市宮川,ＪＲ中央線/茅野駅 歩17分,,,築15年,2階建,\r1階,5.8万円,5100円,-,NaN,1LDK,42.63m2,8.7万円
4,ＪＲ中央線 茅野駅 2階建 築15年,長野県茅野市宮川,ＪＲ中央線/茅野駅 歩17分,,,築15年,2階建,\r2階,6.5万円,5100円,-,NaN,2LDK,54.67m2,9.75万円
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
823,レオパレスやまびこ,長野県茅野市玉川,ＪＲ中央線/茅野駅 バス15分 (バス停)中央病院 歩7分,,,築22年,2階建,2階,4.3万円,5000円,-,NaN,1K,23.18m2,4.3万円
824,レオパレスディアコート,長野県茅野市豊平,ＪＲ中央線/茅野駅 バス9分 (バス停)鬼場 歩7分,,,築22年,2階建,1階,4.1万円,5000円,-,NaN,1K,23.18m2,4.1万円
825,レオパレスディアコート,長野県茅野市豊平,ＪＲ中央線/茅野駅 バス9分 (バス停)鬼場 歩7分,,,築22年,2階建,2階,4.2万円,5000円,-,NaN,1K,23.18m2,4.2万円
826,レオパレスＴ＆Ｍコート,長野県茅野市玉川,ＪＲ中央線/茅野駅 バス22分 (バス停)北久保 歩3分,,,築22年,2階建,2階,3.6万円,5000円,-,NaN,1K,23.18m2,-


In [6]:
print(name)
print(address)
print(station1)
print(station2)
print(station3)
print(building_age)
print(number_of_floors)
print(floor)
print(rent)
print(maintenance_fee)
print(deposit)
print(gratuity)
print(layout)
print(area)

ブルック　ウエスト　II
長野県茅野市宮川
ＪＲ中央線/茅野駅 歩16分


築11年
2階建
1階
6.5万円
3300円
-
9.75万円
1LDK
46.24m2


In [ ]:
# ページ情報の取得テスト


In [16]:
pages = soup.find("ol", class_="pagination-parts")
page = pages.find_all("li")[-1].text
page

'6'